### Обзор попробованных методов
1. Первое решение у меня было fine-tuning только линейного слоя efficientnet_b0 (Public LB 0.93)
2. Дальше я пробовал различные Sharpness or Blur Metrics (в скобках я указал ROC AUC, который получался при применении их к трейну так, как было сделано в бейзлайне):

- статистики на основе edges от фильтров Sobel, Candy, Robert (from skimage.filters import laplace, sobel, roberts)
- Laplasian variance (0.93)
- tenengrad, modified_laplacian, focused_sum
- Haar Wavelet Transform (0.88) - https://github.com/pedrofrodenas/blur-Detection-Haar-Wavelet
- DCT algorithm https://github.com/linghu8812/blur_detector
- Singular Value Decomposition (SVD) https://github.com/fled/blur_detection
- skimage.measure.blur_effect (**0.966**)
- cpbd (0.93) https://github.com/daniilkk/python-cpbd
- No-reference blurred image quality assessment by structural similarity index (0.916) https://github.com/ISipi/nssim_implementation
- Sharpness Estimation for Document and Scene Images (**0.97 !**) https://github.com/umang-singhal/pydom

Отобрав лучшие из них, с помощью catboost получил Public LB 0.9797

3. Вернулся к efficientnet_b0, только тренировал ее вместе с лучшими метриками из предыдущего пункта (Public LB 0.992)
4. Разбивал каждое изображение на 5 частей и усреднял предсказания модели (Public LB 0.987)
5. В итоге убрал метрики, разбиение изображения и тренировал полностью всю efficientnet_b0 - это решение представлено в данном ноутбуке (Public LB 0.996).

efficientnet_b0 - потому что нам нужно хорошее выделение признаков, но одновременно не самая сильная сеть для уменьшения переобучения, так как сложность данной задачи намного меньше той, на которой обучалась efficientnet_b0.

In [1]:
import sys
import numpy as np
import pandas as pd

import os
import gc
import matplotlib.pyplot as plt
import importlib
import pickle

from tqdm.notebook import tqdm

pd.set_option('display.max_rows', 200)
pd.set_option("max_colwidth", 45)
pd.set_option("display.precision", 1)
pd.options.display.float_format = "{:.3f}".format
# pd.set_option("display.max_rows", 5)
# pd.reset_option("display.max_rows")

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

# from pandarallel import pandarallel
# pandarallel.initialize(progress_bar=True)

SEED = 34
N_CPU = os.cpu_count()

In [2]:
import cv2
import random
from torch.utils.data import DataLoader,Dataset
from typing import Union, Any, Optional, Tuple, Dict, List
import torchvision.transforms as T
import torchvision
from PIL import Image

import torch
from torch import nn, Tensor
from torch.nn.modules.loss import BCEWithLogitsLoss
from torchmetrics.classification import BinaryAUROC
import torch.optim as optim

torch.__version__, torchvision.__version__

('1.11.0', '0.12.0')

In [3]:
dir_data = '/kaggle/input/shift-cv-winter-2023/'

DIR_TRAIN = dir_data + 'train/train/'
DIR_TEST = dir_data + 'test/test/'

df_train = pd.read_csv(dir_data + 'train.csv')
df_test = pd.read_csv(dir_data + 'sample_submission.csv')

df_train['filename'] = DIR_TRAIN + df_train['filename']
df_test['filename'] = DIR_TEST + + df_test['filename']
df_train[:3]

,filename,blur
0,/kaggle/input/shift-cv-winter-2023/train/...,0.000
1,/kaggle/input/shift-cv-winter-2023/train/...,0.000
2,/kaggle/input/shift-cv-winter-2023/train/...,0.000


In [4]:
df_train['blur'].value_counts()

0.000    1367
1.000    1297
Name: blur, dtype: int64

In [6]:
def add_folds(df:pd.DataFrame, n_folds:int=4, seed:int=34) -> pd.DataFrame:

    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)

    df['fold'] = -1
    for fold, (trn_, val_) in enumerate(skf.split(df,df['blur'])):
        df.loc[val_,'fold'] = fold

    return df

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

def set_seed(seed: int = 34) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

set_seed(SEED)

cuda
Random seed set as 34


In [8]:
class Dset(Dataset):
    def __init__(self, df:pd.DataFrame, augmentation:Optional[T.Compose]=None):
        
        self.df = df.reset_index(drop=True)
        self.labels = torch.tensor(self.df['blur'].to_numpy(), dtype=torch.float32).unsqueeze(1)
        self.aug = augmentation

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):

        img_path = self.df['filename'].iloc[idx]
        image = Image.open(img_path)

        if self.aug:
            image = self.aug(image)

        return {
            "image": image,
            "label": self.labels[idx],
            }

In [9]:
OUTPUT_SHAPE = (224, 224)

train_transform = T.Compose([
    T.RandomCrop(OUTPUT_SHAPE), 
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    T.RandomVerticalFlip(p=0.5),
    T.RandomHorizontalFlip(p=0.5)]) 

val_transform = T.Compose([
    T.CenterCrop(OUTPUT_SHAPE), 
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [10]:
class Model(nn.Module):
    def __init__(self, hid_dim:int = 10) -> None:
        super().__init__()

        self.feats_extractor = torchvision.models.efficientnet_b0(pretrained=True)
        self.feats_extractor.classifier = nn.Sequential(
            torch.nn.Dropout(p=0.4, inplace=True), 
            torch.nn.Linear(in_features=1280, out_features=hid_dim),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.1), 
            torch.nn.Linear(in_features=hid_dim, out_features=1))

    def forward(self, batch:Dict[str,Tensor]):
        x = self.feats_extractor(batch['image'])
        return x

class Trainer():
    def __init__(self, model: torch.nn.Module, train_dataloader: DataLoader, val_dataloader: DataLoader,loss_fn: torch.nn.Module, optimizer: torch.optim.Optimizer,scheduler, device: Union[torch.device,str], max_tol:int=14, load_best:bool=True, fold:int=111) -> None:

        self.model = model.to(device)
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device

        self.train_dl = train_dataloader
        self.val_dl = val_dataloader

        self.loss_fn = loss_fn
        self.metric = BinaryAUROC()

        self.train_losses = []
        self.val_losses = []

        self.best_metric = None
        self.max_tol = max_tol
        self.tol = 0

        self.total_epochs = 0
        self.load_best = load_best
        self.best_ckpt = None
        self.fold = fold
        
        self.ckpt_dir = 'ckpts/'    
        os.makedirs(self.ckpt_dir,exist_ok=True)

    def training_step(self, batch:Dict[str,Tensor]) -> None:
        batch = self.dict_to_device(batch)
        y = batch['label']

        y_pred = self.model(batch)
        loss = self.loss_fn(y_pred, y)

        loss.backward()
        self.optimizer.step()
        self.optimizer.zero_grad()

        self.train_losses.append(loss.item())

    def validation_step(self, batch:Dict[str,Tensor]) -> None:
        batch = self.dict_to_device(batch)
        y = batch['label']

        y_pred = self.model(batch)
        loss = self.loss_fn(y_pred, y)
        
        #  If preds has values outside [0,1] range we consider the input to be logits and will auto apply sigmoid per element.
        self.metric(y_pred,y)

        self.val_losses.append(loss.item())

    def train(self, max_epochs:int=1000) -> None:

        for epoch in tqdm(range(max_epochs)):
            self.total_epochs += 1

            self.model.train()
            for batch in tqdm(self.train_dl, leave=False):
                self.training_step(batch)

            self.model.eval()
            with torch.inference_mode():
                for batch in self.val_dl:
                    self.validation_step(batch)

            train_epoch_loss = np.mean(self.train_losses)
            val_epoch_loss = np.mean(self.val_losses)
            val_metric = self.metric.compute()
            self.reset_losses_metrics()

            self.scheduler.step(val_metric)

            print(f"Epoch: {epoch+1} | "
                    f"train_loss: {train_epoch_loss:.4f} | "
                    f"val_loss: {val_epoch_loss:.4f} | "
                    f"ROC_AUC: {val_metric:.5f}")

            if self.early_stopping(val_metric):
                break
            elif self.is_best_metric(val_metric):
                self.save_checkpoint(f'ckpts/best_fold_{self.fold}.ckpt')
            

        print(f'[BEST_METRIC]: {self.best_metric}')

        if self.load_best:
            self.load_checkpoint(self.best_ckpt)

    def predict(self, test_dl:DataLoader):
        preds = []
        self.model.eval()
        with torch.inference_mode():
            for batch in test_dl:
                batch = self.dict_to_device(batch)
                y_probs = torch.sigmoid(self.model(batch)).cpu()
                preds.append(y_probs)

        return torch.cat(preds).numpy()

    def dict_to_device(self, batch:Dict[str,torch.tensor]) -> Dict[str,torch.tensor]:
        return {k: v.to(self.device) if hasattr(v, 'to') else v for k, v in batch.items()}

    def reset_losses_metrics(self) -> None:
        self.train_losses = []
        self.val_losses = []
        self.metric.reset()

    def is_best_metric(self, val_metric:float) -> bool:
        if (self.best_metric is None) or val_metric > self.best_metric:
            self.best_metric = val_metric
            return True
        return False

    def early_stopping(self, val_metric:float) -> bool:
        if (self.best_metric is not None) and val_metric <= self.best_metric:
            self.tol += 1
        else:
            self.tol = 0
        if self.tol >= self.max_tol:
            print('early_stopping')
            return True
        return False

    def save_checkpoint(self, filename:str = 'best.ckpt') -> None:
        checkpoint = {'epoch': self.total_epochs,
                      'model_state_dict': self.model.state_dict(),
                      'optimizer_state_dict': self.optimizer.state_dict(),
                      'best_metric':self.best_metric}
        torch.save(checkpoint, filename)
        self.best_ckpt = filename

    def load_checkpoint(self, filename:str = 'best.ckpt') -> None:
        print('load checkpoint: ', filename)
        checkpoint = torch.load(filename)

        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.model.train()

In [ ]:
# import shutil
# shutil.rmtree('./ckpts')

In [11]:
from itertools import product

N_REPEATS = 1
N_FOLDS = 4
TRAIN_BATCH_SIZE = 64
VAL_BATCH_SIZE = 128

test_ds = Dset(df_test, augmentation=val_transform)
test_dl = DataLoader(test_ds, batch_size=VAL_BATCH_SIZE, num_workers=N_CPU)

preds = np.zeros(len(df_test))

for repeat, fold in product(range(N_REPEATS),range(N_FOLDS)):
    
    print(f'\n[START TRAINING]: REPEAT {repeat} FOLD {fold}')
    df_train = add_folds(df_train, n_folds=N_FOLDS, seed=SEED*(repeat+1))

    Xy_train = df_train[df_train['fold']!=fold]
    Xy_val = df_train[df_train['fold']==fold]

    train_ds = Dset(Xy_train, augmentation=train_transform)
    val_ds = Dset(Xy_val, augmentation=val_transform)

    train_dl = DataLoader(train_ds, batch_size=TRAIN_BATCH_SIZE, shuffle=True, num_workers=N_CPU)
    val_dl = DataLoader(val_ds, batch_size=VAL_BATCH_SIZE, num_workers=N_CPU)

    model = Model().to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor=0.5, patience=5, threshold=0.0001, min_lr=0.00005,verbose=True)

    loss_fn = BCEWithLogitsLoss()

    trainer = Trainer(model=model,
                       train_dataloader=train_dl,
                       val_dataloader=val_dl,
                       optimizer=optimizer,
                       scheduler=scheduler,
                       loss_fn=loss_fn,
                       device=device,
                       fold=f'{repeat}_{fold}')

    trainer.train(max_epochs=1000)

    fold_preds = trainer.predict(test_dl)
    preds += fold_preds.flatten()

    del model, trainer
    gc.collect()

    torch.cuda.empty_cache()


[START TRAINING]: REPEAT 0 FOLD 0


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth


  0%|          | 0.00/20.5M [00:00<?, ?B/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

...

Epoch: 62 | train_loss: 0.0198 | val_loss: 0.0308 | ROC_AUC: 0.99939


  0%|          | 0/32 [00:00<?, ?it/s]

Epoch: 63 | train_loss: 0.0154 | val_loss: 0.0332 | ROC_AUC: 0.99927


  0%|          | 0/32 [00:00<?, ?it/s]

Epoch: 64 | train_loss: 0.0173 | val_loss: 0.0321 | ROC_AUC: 0.99929
early_stopping
[BEST_METRIC]: 0.9994586110115051
load checkpoint:  ckpts/best_fold_0_3.ckpt


In [13]:
sub = pd.DataFrame()
sub['filename'] = df_test['filename'].apply(lambda x : os.path.split(x)[1])
sub['blur'] = pd.Series(preds).round(5)
sub.to_csv('submission18.csv', index=False)
sub[:3]

,filename,blur
0,bnxzvzqlzlnnbxfkcuin.jpg,0.001
1,powqsnpoynygwqsciedp.jpg,2.041
2,zpjlbfhurhygjnqccpii.jpg,0.005
